<a href="https://colab.research.google.com/github/Meerasrilekha/Vehicle-Cut-in-detection---Code-Crusaders/blob/main/Vehicle_cut_in_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#After re-encoding the vehicle input video final_video_processing function processes the final cut-in detection video
from moviepy.editor import VideoFileClip
#function starts here with parameters of the input video file and csv file which contains vehicle dataset
def final_video_processing(video_file, ground_truth_csv):
  #path for the csv file
    ground_truth_data = read_ground_truth_from_csv(ground_truth_csv)
    video_clip = VideoFileClip(video_file)
    #splits the video into different frames and evaluates each frame
    frame_list = [frame for frame in video_clip.iter_frames()]
    all_detected_vehicles = []
    all_ground_truth_vehicles = []
    #
    for frame_number, frame in enumerate(frame_list):
    # Iterate through each frame in the frame_list
    # frame_number is the index of the current frame
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        # Convert the frame from RGB to BGR color space, as required by OpenCV
        final_frame = pipeline(frame_rgb)
        # Pass the BGR frame through a pipeline (likely a machine learning model) and to get the final processed frame
        ground_truth_vehicles = []
        for label in ground_truth_data:
            ground_truth_vehicles.extend(ground_truth_data[label])
        # Collect all the ground truth vehicle locations from the ground_truth_data
        # The ground_truth_data is likely a dictionary, where the keys are labels and the values are lists of vehicle locations

        detected_vehicles = detect_vehicles(final_frame)
        # Use a vehicle detection algorithm to detect vehicles in the final_frame and the detected_vehicles is a list of detected vehicle locations

        all_detected_vehicles.extend(detected_vehicles)
        all_ground_truth_vehicles.extend(ground_truth_vehicles)
    # Accumulate all the detected and ground truth vehicles across all frames

    tp, fp, fn = evaluate_detection(all_detected_vehicles, all_ground_truth_vehicles)
    # Evaluate the detection performance by comparing the detected vehicles
    # true positives (tp), false positives (fp), and false negatives (fn)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    # Calculate the precision, which is the ratio of true positives to
    # the sum of true positives and false positives
    # If there are no true positives and false positives, the precision is 0

    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    # Calculate the recall, which is the ratio of true positives to
    # the sum of true positives and false negatives
    # If there are no true positives and false negatives, the recall is 0

    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    # Calculate the F1-score, which is the harmonic mean of precision and recall
    # The F1-score is a balanced metric that considers both precision and recall
    # If the sum of precision and recall is 0, the F1-score is also 0
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

    output_file = 'result.mp4'
    final_clip = VideoFileClip(video_file).fl_image(lambda img: pipeline(cv2.cvtColor(img, cv2.COLOR_RGB2BGR)))
    final_clip.write_videofile(output_file, codec='libx264')
#function calling and final output processing
if __name__ == "__main__":
    vehicle_input_video = "New_Project.mp4"
    vehicle_input_trail = "New_Project.mp4"
    ground_truth_csv_path = "ground_truth_csv.csv"
    reencode_video(vehicle_input_video,vehicle_input_trail)
    print(ground_truth_csv_path)
    final_video_processing(vehicle_input_trail, ground_truth_csv_path)

ground_truth_csv.csv
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
Moviepy - Building video result.mp4.
MoviePy - Writing audio in resultTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video result.mp4



Moviepy - Done !
Moviepy - video ready result.mp4


In [ ]:
import cv2
import numpy as np
import math
import subprocess

def reencode_video(input_file, output_file):
    """
    Reencode a video file using FFmpeg.
    This function is used to convert the input video file to a format that can be easily processed.
    """
    cmd = f"ffmpeg -i {input_file} -c:v copy -c:a copy -movflags faststart {output_file}"
    subprocess.call(cmd, shell=True)

def region_of_interest(img, vertices):
    """
    Apply a mask to the input image to focus on a specific region of interest.
    This function is used to select the region of the image where the vehicle detection will be performed.
    """
    mask = np.zeros_like(img)
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    """
    Draw lines on the input image.
    This function is used to visualize the detected lane lines on the image.
    """
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    img = np.copy(img)
    if lines is None:
        return img
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    return img

import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

def detect_vehicles(image):
    """
    Detect vehicles in the input image using the YOLOv5 model.
    This function returns a list of detected vehicles, where each vehicle is represented as a tuple (x, y, w, h).
    """
    results = model(image)
    detections = results.pandas().xyxy[0]

    vehicles_detected = []
    for _, row in detections.iterrows():
        if row['name'] in ["car", "bus", "truck", "motorcycle", "scooty", "bike"]:
            x, y, w, h = int(row['xmin']), int(row['ymin']), int(row['xmax']) - int(row['xmin']), int(row['ymax']) - int(row['ymin'])
            vehicles_detected.append((x, y, w, h))
            label = row['name']
            color = (0, 0, 255)
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return vehicles_detected

def highlight_vehicle_area(image, left_line, right_line, vehicles_detected):
    """
    Highlight the area where vehicles are detected and check for vehicle cut-in.
    This function draws a green polygon around the detected vehicle area and displays a warning if a vehicle is detected cutting into the lane.
    """
    overlay = image.copy()
    warning = False
    if left_line is not None and right_line is not None:
        height = image.shape[0]
        min_y = int(height * (3 / 5))
        max_y = int(height)

        left_x_start, left_y_start, left_x_end, left_y_end = left_line
        right_x_start, right_y_start, right_x_end, right_y_end = right_line

        pts = np.array([
            [left_x_start, left_y_start],
            [left_x_end, left_y_end],
            [right_x_end, right_y_end],
            [right_x_start, right_y_start]
        ], np.int32)

        cv2.fillPoly(overlay, [pts], (0, 255, 0))
        cv2.addWeighted(overlay, 0.3, image, 0.7, 0, image)

        for (x, y, w, h) in vehicles_detected:
            vehicle_center = (float(x + w // 2), float(y + h // 2))
            if cv2.pointPolygonTest(pts, vehicle_center, False) >= 0:
                warning = True

    if warning:
        cv2.putText(image, "Warning!!!!! Vehicle cut-in detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    return image

def pipeline(image):
    """
    The main processing pipeline for the vehicle cut-in detection.
    This function takes an input image, processes it, and returns the final image with the detected vehicles and the warning for vehicle cut-in.
    """
    height = image.shape[0]
    width = image.shape[1]
    region_of_interest_vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height),
    ]
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    cannyed_image = cv2.Canny(gray_image, 100, 200)

    cropped_image = region_of_interest(cannyed_image, np.array([region_of_interest_vertices], np.int32))

    lines = cv2.HoughLinesP(cropped_image, rho=6, theta=np.pi / 60, threshold=160, lines=np.array([]), minLineLength=20, maxLineGap=300)

    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []

    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x2 - x1 == 0:  # Prevent division by zero
                    continue
                slope = (y2 - y1) / (x2 - x1)
                if math.fabs(slope) < 0.5:
                    continue
                if slope <= 0:
                    left_line_x.extend([x1, x2])
                    left_line_y.extend([y1, y2])
                else:
                    right_line_x.extend([x1, x2])
                    right_line_y.extend([y1, y2])

    left_line = None
    right_line = None
    if left_line_x and left_line_y and right_line_x and right_line_y:
        min_y = int(image.shape[0] * (3 / 5))
        max_y = int(image.shape[0])

        if left_line_x and left_line_y:
            poly_left = np.poly1d(np.polyfit(left_line_y, left_line_x, deg=1))
            left_line = (int(poly_left(max_y)), max_y, int(poly_left(min_y)), min_y)

        if right_line_x and right_line_y:
            poly_right = np.poly1d(np.polyfit(right_line_y, right_line_x, deg=1))
            right_line = (int(poly_right(max_y)), max_y, int(poly_right(min_y)), min_y)

        line_image = draw_lines(image, [[left_line, right_line]], thickness=5)
    else:
        line_image = image  # If no lines are detected, return the original image

    vehicles_detected = detect_vehicles(line_image)

    final_image = highlight_vehicle_area(line_image, left_line, right_line, vehicles_detected)

    return final_image

def calculate_iou(boxA, boxB):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    This function is used to evaluate the detection performance by comparing the detected vehicles with the ground truth vehicles.
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def evaluate_detection(detected_vehicles, ground_truth_vehicles, iou_threshold=0.5):
    """
    Evaluate the detection performance by comparing the detected vehicles with the ground truth vehicles.
    This function returns the number of true positives, false positives, and false negatives.
    """
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    matched_ground_truths = []

    for gt_vehicle in ground_truth_vehicles:
        gt_box = (gt_vehicle[0], gt_vehicle[1], gt_vehicle[0] + gt_vehicle[2], gt_vehicle[1] + gt_vehicle[3])
        matched = False
        for det_vehicle in detected_vehicles:
            det_box = (det_vehicle[0], det_vehicle[1], det_vehicle[0] + det_vehicle[2], det_vehicle[1] + det_vehicle[3])
            if calculate_iou(gt_box, det_box) >= iou_threshold:
                matched = True
                break

        if matched:
            true_positives += 1
        else:
            false_negatives += 1

    false_positives = len(detected_vehicles) - true_positives

    return true_positives, false_positives, false_negatives

def read_ground_truth_from_csv(ground_truth_csv):
    """
    Read the ground truth vehicle locations from a CSV file.
    This function returns a dictionary where the keys are the labels and the values are lists of vehicle bounding boxes.
    """
    import pandas as pd
    df = pd.read_csv(ground_truth_csv)
    if 'labels' not in df.columns:
        raise ValueError("CSV file must contain a 'labels' column.")
    if 'x_min' not in df.columns or 'y_min' not in df.columns or 'x_max' not in df.columns or 'y_max' not in df.columns:
        raise ValueError("CSV file must contain 'x_min', 'y_min', 'x_max', and 'y_max' columns.")

    labels = df['labels'].unique()
    ground_truth_data = {label: [] for label in labels}
    for _, row in df.iterrows():
        label = row['labels']
        x_min = int(row['x_min'])
        y_min = int(row['y_min'])
        x_max = int(row['x_max'])
        y_max = int(row['y_max'])
        box = (x_min, y_min, x_max - x_min, y_max - y_min)
        ground_truth_data[label].append(box)
    return ground_truth_data


/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 201.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 164.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 12.3s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-7-14 Python-3.10.12 torch-2.3.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 127MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
import cv2
import torch
import numpy as np
from collections import deque

# Load YOLOv5 model from PyTorch Hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Initialize tracker
class SimpleTracker:
    def __init__(self):
        self.trackers = []

    def update(self, detections):
        """
        Update the tracker with the latest detections.
        This function takes a list of detections (bounding boxes) and updates the existing trackers.
        It returns the updated list of trackers.
        """
        for tracker in self.trackers:
            tracker['lost'] += 1

        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            matched = False
            for tracker in self.trackers:
                if self.iou(tracker['bbox'], [x1, y1, x2, y2]) > 0.5:
                    tracker['bbox'] = [x1, y1, x2, y2]
                    tracker['lost'] = 0
                    matched = True
                    break
            if not matched:
                self.trackers.append({'bbox': [x1, y1, x2, y2], 'lost': 0})

        self.trackers = [t for t in self.trackers if t['lost'] < 5]

        return self.trackers

    def iou(self, bbox1, bbox2):
        """
        Calculate the Intersection over Union (IoU) of two bounding boxes.
        This function is used to determine if a detection matches an existing tracker.
        """
        x1, y1, x2, y2 = bbox1
        x1_, y1_, x2_, y2_ = bbox2

        xi1, yi1 = max(x1, x1_), max(y1, y1_)
        xi2, yi2 = min(x2, x2_), min(y2, y2_)

        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

        bbox1_area = (x2 - x1) * (y2 - y1)
        bbox2_area = (x2_ - x1_) * (y2_ - y1_)

        union_area = bbox1_area + bbox2_area - inter_area

        return inter_area / union_area

# Calculate Euclidean distance between two vehicles
def calculate_distance(vehicle1, vehicle2):
    """
    Calculate the Euclidean distance between the centers of two vehicles.
    This function is used to determine if two vehicles are too close to each other.
    """
    pos1 = np.array(vehicle1['center'])
    pos2 = np.array(vehicle2['center'])
    distance = np.linalg.norm(pos1 - pos2)
    return distance

# Process video frames
def process_video(video_path):
    """
    Process the input video and detect vehicles, track them, and check for vehicle cut-in warnings.
    This function reads the video frames, processes them using the YOLOv5 model and the SimpleTracker,
    and writes the output video with the detected vehicles and the warning messages.
    """
    cap = cv2.VideoCapture(video_path)
    tracker = SimpleTracker()
    vehicle_tracks = {}
    warning_distance = 50  # Threshold distance in pixels

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('warnings.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        detections = []
        for *box, conf, cls in results.xyxy[0].cpu().numpy():
            if cls == 2:  # Only consider cars
                detections.append(box + [conf, cls])

        tracks = tracker.update(detections)

        for i, track in enumerate(tracks):
            x1, y1, x2, y2 = map(int, track['bbox'])
            center = [(x1 + x2) / 2, (y1 + y2) / 2]
            track['center'] = center
            if i not in vehicle_tracks:
                vehicle_tracks[i] = deque(maxlen=5)
            vehicle_tracks[i].append(center)
            if len(vehicle_tracks[i]) > 1:
                track['velocity'] = np.mean(np.diff(vehicle_tracks[i], axis=0), axis=0)
            else:
                track['velocity'] = np.array([0, 0])

            # Draw bounding box and center
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.circle(frame, (int(center[0]), int(center[1])), 2, (0, 0, 255), -1)
            cv2.putText(frame, f'ID: {i}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        for i, track in enumerate(tracks):
            for j, other_track in enumerate(tracks):
                if i != j:
                    distance = calculate_distance(track, other_track)
                    if distance < warning_distance:
                        cv2.putText(frame, f'Warning! Vehicles {i} and {j} too close!', (50, 50 + 30*i), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        out.write(frame)

    cap.release()
    out.release()

if __name__ == "__main__":
    video_path ='New_Project.mp4'  # Replace with your local video file path
    process_video(video_path)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-14 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
